# Neural Prohpet
newpycaret env

https://github.com/ourownstory/neural_prophet
https://neuralprophet.com/how-to-guides/feature-guides/global_local_modeling.html


In [18]:
# Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import snowflake.connector
import os
from sklearn.impute import SimpleImputer
from neuralprophet import NeuralProphet, set_log_level
from datetime import datetime, timedelta
from dateutil import relativedelta
from autots import AutoTS

2. Load Data
_______________________________________________________________

### To Do: copy and paste in to a new chunk, enter credentials and run to save in environment. Then delete chunk
%env snowflakeuser=<your_snowflake_username> <br>
%env snowflakepass=<your_snowflake_password>

In [2]:
%env snowflakeuser=Crudek
%env snowflakepass=Vitamix22!

env: snowflakeuser=Crudek
env: snowflakepass=Vitamix22!


In [5]:
# Query Snowflake

# Snowflake connection parameters
connection_params = {
    "user": os.environ['snowflakeuser'],
    "password": os.environ['snowflakepass'],
    "account": "zib52348.us-east-1",
    "role": "ACCOUNTADMIN",
    "warehouse": "REPORTING",
    "database": "ANALYTICS",
    "schema": "SALES",
}

# Establish a connection to Snowflake
conn = snowflake.connector.connect(**connection_params)

with open('costco_e520_query.sql', 'r') as query:
    # connection == the connection to your database, in your case prob_db
    df_initial = pd.read_sql_query(query.read(), conn)

# Close the connection
conn.close()

WARNING - (py.warnings._showwarnmsg) - C:\Users\crudek\AppData\Local\Temp\ipykernel_3696\566721061.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_initial = pd.read_sql_query(query.read(), conn)



In [6]:
df_initial

,MONTH,UNITS
0,2022-12-01,25628.0
1,2023-02-01,8201.0
2,2022-11-01,21570.0
3,2023-06-01,2990.0
4,2022-09-01,4794.0
5,2022-10-01,9913.0
6,2023-09-01,3855.0
7,2023-08-01,5905.0
8,2023-05-01,14365.0
9,2022-08-01,3027.0


In [9]:
# create copy of df_d
df = df_initial.copy(deep=True)

# convert month field to date
df["MONTH"] = pd.to_datetime(df["MONTH"])

# Some random months will have data that we want to remove (* Want to test without July though)
df = df.sort_values(['MONTH'])  # reorder dataframe

# create series
df_s = df.set_index(['MONTH'])['UNITS']
# convert back to dataframe
df_d = df_s.to_frame()
#reset index
df_d.reset_index(inplace=True)

df_d2 = df_d[df_d['MONTH'] < '2023-10-01']

In [16]:
data = df_d2.rename(columns={'MONTH': 'ds', 'UNITS': 'y'})
    
# Initialize and train the model
model = NeuralProphet()
model.fit(data, freq="MS")

# Create a dataframe with future dates for prediction
future = model.make_future_dataframe(data, periods=3, n_historic_predictions=len(data))
    
# Make predictions for the next 12 periods
forecast = model.predict(future)

WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to [92.857]% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - MS
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling weekly seasonality. Run NeuralProphet with weekly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 14
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 1000
WARNING - (NP.config.set_lr_f

Finding best initial lr:   0%|          | 0/201 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to [92.857]% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - MS
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to [94.118]% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - MS
INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to [94.118]% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - MS


Predicting: 1it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


In [19]:
# model
model = AutoTS(
    forecast_length = 3,
    frequency = 'infer',
    ensemble = 'simple', 
    models_mode = 'deep',
    model_list = 'univariate',
    max_generations = 10,
    num_validations = 10,
    no_negatives = True,
    n_jobs = 'auto'
)

model = model.fit(
    data,
    date_col = 'ds',
    value_col = 'y',
)

# print(model)

# create prediction
prediction = model.predict(forecast_length=3)

# temp fcast dataframe
temp_fcasts = prediction.forecast

Using 6 cpus for n_jobs.
Data frequency is: MS, used frequency is: MS
Too many training validations for length of data provided, decreasing num_validations to 3
Model Number: 1 with model ARIMA in generation 0 of 10
Model Number: 2 with model AverageValueNaive in generation 0 of 10
Model Number: 3 with model AverageValueNaive in generation 0 of 10
Model Number: 4 with model AverageValueNaive in generation 0 of 10
Model Number: 5 with model DatepartRegression in generation 0 of 10
Model Number: 6 with model DatepartRegression in generation 0 of 10
Model Number: 7 with model DatepartRegression in generation 0 of 10
Model Number: 8 with model DatepartRegression in generation 0 of 10
Template Eval Error: ImportError('Tensorflow not available, install with pip install tensorflow.') in model 8 in generation 0: DatepartRegression
Model Number: 9 with model ETS in generation 0 of 10
Model Number: 10 with model ETS in generation 0 of 10
ETS error ValueError('Cannot compute initial seasonals usi

WARNING - (py.warnings._showwarnmsg) - c:\Users\crudek\AppData\Local\anaconda3\envs\newpycaret\lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1257: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)



WARNING - (py.warnings._showwarnmsg) - c:\Users\crudek\AppData\Local\anaconda3\envs\newpycaret\lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1257: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)

WARNING - (py.warnings._showwarnmsg) - c:\Users\crudek\AppData\Local\anaconda3\envs\newpycaret\lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1257: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)



Model Number: 12 with model GLM in generation 0 of 10
Model Number: 13 with model GLS in generation 0 of 10
Model Number: 14 with model GLS in generation 0 of 10
Model Number: 15 with model LastValueNaive in generation 0 of 10
Model Number: 16 with model LastValueNaive in generation 0 of 10
Model Number: 17 with model LastValueNaive in generation 0 of 10
Model Number: 18 with model LastValueNaive in generation 0 of 10
Model Number: 19 with model SeasonalNaive in generation 0 of 10


WARNING - (py.warnings._showwarnmsg) - c:\Users\crudek\AppData\Local\anaconda3\envs\newpycaret\lib\site-packages\autots\tools\probabilistic.py:67: RuntimeWarning: invalid value encountered in divide
  (prior_mu / prior_sigma**2) + ((n * data_mu) / prior_sigma**2)

WARNING - (py.warnings._showwarnmsg) - c:\Users\crudek\AppData\Local\anaconda3\envs\newpycaret\lib\site-packages\autots\tools\probabilistic.py:68: RuntimeWarning: divide by zero encountered in divide
  ) / ((1 / prior_sigma**2) + (n / prior_sigma**2))



...

142 - FBProphet with avg smape 119.01: 
Model Number: 143 of 143 with model FBProphet for Validation 2


14:59:15 - cmdstanpy - INFO - Chain [1] done processing


143 - FBProphet with avg smape 64.28: 
Validation Round: 3
Model Number: 1 of 143 with model Ensemble for Validation 3
Template Eval Error: ValueError('Need at least 3 dates to infer frequency') in model 1 in generation 0: Ensemble
Model Number: 2 of 143 with model Ensemble for Validation 3
Template Eval Error: ValueError('Need at least 3 dates to infer frequency') in model 2 in generation 0: Ensemble
Model Number: 3 of 143 with model Ensemble for Validation 3
Template Eval Error: ValueError('Need at least 3 dates to infer frequency') in model 3 in generation 0: Ensemble
Model Number: 4 of 143 with model Ensemble for Validation 3
Template Eval Error: ValueError('Need at least 3 dates to infer frequency') in model 4 in generation 0: Ensemble
Model Number: 5 of 143 with model Ensemble for Validation 3
Template Eval Error: ValueError('Need at least 3 dates to infer frequency') in model 5 in generation 0: Ensemble
Model Number: 6 of 143 with model Ensemble for Validation 3
Template Eval Er

WARNING - (py.warnings._showwarnmsg) - c:\Users\crudek\AppData\Local\anaconda3\envs\newpycaret\lib\site-packages\autots\models\ensemble.py:1021: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  cur_mods = pd.Series(models_pos).value_counts()

WARNING - (py.warnings._showwarnmsg) - c:\Users\crudek\AppData\Local\anaconda3\envs\newpycaret\lib\site-packages\autots\models\ensemble.py:1021: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  cur_mods = pd.Series(models_pos).value_counts()

WARNING - (py.warnings._showwarnmsg) - c:\Users\crudek\AppData\Local\anaconda3\envs\newpycaret\lib\site-packages\autots\models\ensemble.py:1021: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to si

Model Number: 1488 with model Ensemble in generation 12 of Ensembles
Template Eval Error: ValueError('BestN failed, no component models available.') in model 1488 in generation 12: Ensemble
Model Number: 1489 with model Ensemble in generation 12 of Ensembles
Template Eval Error: ValueError('BestN failed, no component models available.') in model 1489 in generation 12: Ensemble
Model Number: 1490 with model Ensemble in generation 12 of Ensembles
Template Eval Error: ValueError('BestN failed, no component models available.') in model 1490 in generation 12: Ensemble
Generation 12 had all new models fail
Validation Round: 1
Model Number: 1 of 3 with model Ensemble for Validation 1
Template Eval Error: ValueError('BestN failed, no component models available.') in model 1 in generation 0: Ensemble
Model Number: 2 of 3 with model Ensemble for Validation 1
Template Eval Error: ValueError('BestN failed, no component models available.') in model 2 in generation 0: Ensemble
Model Number: 3 of 3 w

In [20]:
temp_fcasts

,y
2023-10-01,5765.746141
2023-11-01,10346.619397
2023-12-01,12986.400000
